In [ ]:
#  count_food2.py

from collections import Counter

import spacy
from spacy.pipeline import EntityRuler
import pandas as pd

# 1. 口コミデータの読み込み
df = pd.read_csv('text/miyajima_sample.csv', encoding='utf-8',
                 parse_dates=['旅行の時期'])

nlp = spacy.load("ja_ginza")

In [ ]:
# 2. 固有表現抽出のルールを追加
patterns = [
    {"label": "Dish", "pattern": [
        {"TEXT": "あなご"},
        {"TEXT": {"REGEX": r"飯|めし"}},
    ]},
    {"label": "Dish", "pattern": "揚げもみじ"},
    {"label": "Dish", "pattern": "ガンス"},
    {"label": "Dish", "pattern": [
        {"TEXT": "にぎり"},
        {"TEXT": "天"}
    ]}
]

ruler = EntityRuler(nlp, overwrite_ents=True)
ruler.add_patterns(patterns)
nlp.add_pipe(ruler,  before="ner")

In [ ]:
# 3. 食べ物に関する単語をリストに追加
food = []
for doc in nlp.pipe(df["口コミ"]):
    # 食べ物に関する固有表現を抽出
    for ent in doc.ents:
        if ent.label_ == "Dish" or ent.label_ == "Food_other":
            food.append(ent.text)

# 4. 食べ物に関する単語の頻度をカウント
counter = Counter(food)

In [ ]:
# 5. 表記ゆれを解消するための用語辞書を用意
# 用語辞書のキーは代表語，値は表記ゆれの単語のリスト
name2variants = {"あなご飯": ["穴子飯", "あなごめし", "穴子めし"],
                 "揚げもみじ": ["揚げ紅葉", "揚げ紅葉饅頭"]}

# 用語辞書を，表記揺れをチェックしやすい形式に変換
variant2name = {}       # キーは表記ゆれの単語，値は代表語
for name, variants in name2variants.items():
    variant2name[name] = name
    for variant in variants:
        variant2name[variant] = name

In [ ]:
# 6. 表記ゆれを解消して，食べ物に関する単語の頻度をカウント
counter2 = {}    # 表記ゆれ解消後の単語の頻度を保存するための辞書
for word, count in counter.most_common():
    if word in variant2name:
        if variant2name[word] in counter2:
            counter2[variant2name[word]] += count
        else:
            counter2[variant2name[word]] = count
    else:
        counter2[word] = count

counter2 = Counter(counter2)

# 7. 食べ物に関する単語の頻度を表示
print('count word')
for word, count in counter2.most_common():
    print(f'{count:>5} {word}')

In [ ]:
#  「男性」の口コミに出現する食べ物に関する単語の頻度をカウント

# 「男性」の口コミに限定
df_man = df[df['性別'] == '男性']['口コミ']

# 食べ物に関する単語をリストに追加
food = []
for doc in nlp.pipe(df_man):
    # 食べ物に関する固有表現を抽出
    for ent in doc.ents:
        if ent.label_ == "Dish" or ent.label_ == "Food_other":
            food.append(ent.text)

# 食べ物に関する単語の頻度をカウント
counter = Counter(food)

# 表記ゆれを解消して，食べ物に関する単語の頻度をカウント
counter2 = {}    # 表記ゆれを解消した後の単語の頻度を保存するための辞書
for word, count in counter.most_common():
    if word in variant2name:
        if variant2name[word] in counter2:
            counter2[variant2name[word]] += count
        else:
            counter2[variant2name[word]] = count
    else:
        counter2[word] = count

counter2 = Counter(counter2)

# 食べ物に関する単語の頻度を表示
print("●　男性の場合")
print('count word')
for word, count in counter2.most_common():
    print(f'{count:>5} {word}')

In [ ]:
# 「女性」の口コミに出現する食べ物に関する単語の頻度をカウント

# 「女性」の口コミに限定
df_man = df[df['性別'] == '女性']['口コミ']

# 食べ物に関する単語をリストに追加
food = []
for doc in nlp.pipe(df_man):
    # 食べ物に関する固有表現を抽出
    for ent in doc.ents:
        if ent.label_ == "Dish" or ent.label_ == "Food_other":
            food.append(ent.text)

# 食べ物に関する単語の頻度をカウント
counter = Counter(food)

# 表記ゆれを解消して，食べ物に関する単語の頻度をカウント
counter2 = {}    # 表記ゆれを解消した後の単語の頻度を保存するための辞書
for word, count in counter.most_common():
    if word in variant2name:
        if variant2name[word] in counter2:
            counter2[variant2name[word]] += count
        else:
            counter2[variant2name[word]] = count
    else:
        counter2[word] = count

counter2 = Counter(counter2)

# 食べ物に関する単語の頻度を表示
print("●　女性の場合")
print('count word')
for word, count in counter2.most_common():
    print(f'{count:>5} {word}')